## CarveMe models 

In [ ]:
import glob
import os
import reframed 
import pandas as pd
import cobra
from pathlib import Path

### Create models with CarveMe on rich and poor media, with and without O2 and with full environment. 

Commands can be found at reconstructingModels notebook

Documentation: https://carveme.readthedocs.io/en/latest/

Original publication: https://academic.oup.com/nar/article/46/15/7542/5042022?login=true

In [ ]:
# path to models
path = "/data/precomputed/sbmlModels/carveme/"

dir_list = sorted(os.listdir(path))
os.chdir(path)
dir_list


In [ ]:
#Bacteroides_thetaiotaomicron anaerobic M9 example
model = load_cbmodel(dir_list[3])
model.summary()

Code: For all models gather general statistics (e.g number of compounds, reactions etc.), run FBA, retrieve media dependent gap-filled reactions. Store everything into a .csv table 

In [ ]:
#FBA with reframed needs solver CPLEX
#create a folder "EX_reactions"
# path_ex = "/root/work/sbmlModels/GAPFILL/EX_reactions"
# F = []
# ex = []
# for file_name in dir_list:
#     GR = []
#     GR_id = []
#     model = load_cbmodel(file_name)
#     solution = FBA(model) 
#     GR.append(file_name)
#     GR.append(solution.fobj) 

#     for c_id in model.compartments:
#         GR.append(len(model.get_compartment_metabolites(c_id)))
#     GR.append(len(model.reactions))
#     check_growth=0# get gap-filled number of reactions
#     gap_fill=-1
#     for c_id in model.reactions:
#         if(check_growth==1):
#             GR_id="%s;%s"%(GR_id,c_id)
#             gap_fill+=1
#         if(c_id=="Growth"):
#             check_growth=1
#     GR.append(gap_fill)
#     GR.append(GR_id)
#     F.append(GR)
#     exdf = pd.DataFrame(solution.values.items()) export reframed FBA solution
#     exdf.to_csv(path_ex+'/'+'/RN_'+file_name.replace(".xml", ".csv" )) 
    

# df = pd.DataFrame(F)
# df.to_csv('/root/work/output/GAPFILL/models_stat.csv') 
dir_list_selection = [dir_list[index] for index in [0, 1]]
dir_list_selection # example select two models.

from reframed import load_cbmodel, to_cobrapy

F = []
ex = []
for file_name in dir_list_selection:
    GR = []
    GR_id = []
    model = load_cbmodel(file_name)
    GR.append(file_name)
    cb_model = to_cobrapy(model)
    obj = cb_model.optimize()
    GR.append(obj.objective_value)

    for c_id in model.compartments:
        GR.append(len(model.get_compartment_metabolites(c_id)))
    GR.append(len(model.reactions))
    check_growth=0# get gap-filled number of reactions
    gap_fill=-1
    for c_id in model.reactions:
        if(check_growth==1):
            GR_id="%s;%s"%(GR_id,c_id)
            gap_fill+=1
        if(c_id=="Growth"):
            check_growth=1
    GR.append(gap_fill)
    GR.append(GR_id)
    F.append(GR)
    
df = pd.DataFrame(F)
df.to_csv('~/models_stat_selection.csv') 

Table with full results can be found at /root/work/output/GAPFILL/DATA/models_stat.tsv

CODE: function to get all reaction without gene association rule.

In [ ]:
def remove_genelessreactions(model):
    geneslessreactions = []
    for i in model.reactions:
        if(i.genes==frozenset()):
            geneslessreactions.append(i)
    return geneslessreactions

CODE: retrieve and store all added reactions without GPR association. 

In [ ]:
#create a folder "Geneless_reactions"
output_path = '~/Geneless_reactions'
Path(output_path).mkdir()

model_ids = []
for filename in dir_list:
    if('gr_'+filename not in os.listdir(output_path) and filename != 'Geneless_removed'):
        model =  cobra.io.read_sbml_model(path+'/'+filename)
        rn = remove_genelessreactions(model)
        RN = pd.DataFrame(rn)
        RN.to_csv(output_path+'/'+'/RN_Geneless_'+filename.replace(".xml", ".csv" )) 


## TASK: explore the generated data from the models (python or R) 

-  The FBA solutions, highlight the two different conditions (O2, -O2) and two media (LB, M9). What do you observe?

-  The number of media dependent gap-filled reactions, highlight the two different conditions (O2, -O2) and two media (LB, M9). What do you observe? 

-  The number of gap-filled reactions (tip! use the files in GAPFILL/Geneless_reactions/)

-  Sub-select all exchange reactions per model and visualize the import and export compounds on both media (tip! use the files in GAPFILL/EX_reactions/ and sub-select reaction based on "R_EX" pattern) 

Try first by yourself! Solution: R-code can be found at /root/work/output/GAPFILL/RCODE/

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3f18ea3e-7409-46ac-b47b-7a847a3fc095' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>